In [1]:
import os
import pandas as pd
import sys
import time

from PIL import Image
from sqlalchemy import create_engine, func
from sqlalchemy.orm import sessionmaker

sys.path.append('../..')
sys.path.append('../../../ajna_docs/commons')


In [2]:
from ajna_commons.flask.conf import SQL_URI
from ajna_commons.utils.images import mongo_image
from virasana.db import mongodb as db


2021-03-03 10:26:45,119 ajna         INFO     Configuração de log efetuada


Fazendo log de erros e alertas no arquivo  ../../../ajna_docs/commons\ajna_commons\flask\error.log
Fazendo log de atividade no arquivo  ../../../ajna_docs/commons\ajna_commons\flask\access.log


In [ ]:
%load_ext autoreload 
%autoreload 2
from virasana.integracao.conformidade_alchemy import Conformidade
from virasana.scripts.conformidadeupdate import completa_conformidade, preenche_isocode
engine = create_engine(SQL_URI)
completa_conformidade(db, engine, 10000)
preenche_isocode(db, engine, 10000)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


2021-03-03 15:42:29,926 ajna         ERROR    (pymysql.err.OperationalError) (1213, 'Deadlock found when trying to get lock; try restarting transaction')
[SQL: UPDATE ajna_conformidade SET tipotrafego=%(tipotrafego)s, vazio=%(vazio)s, paisdestino=%(paisdestino)s, last_modified=CURRENT_TIMESTAMP WHERE ajna_conformidade.`ID` = %(ajna_conformidade_ID)s]
[parameters: {'tipotrafego': 'lci', 'vazio': 0, 'paisdestino': '', 'ajna_conformidade_ID': 17160}]
(Background on this error at: http://sqlalche.me/e/13/e3q8)
Traceback (most recent call last):
  File "C:\Users\25052288840\Anaconda3\lib\site-packages\sqlalchemy\engine\base.py", line 1276, in _execute_context
    self.dialect.do_execute(
  File "C:\Users\25052288840\Anaconda3\lib\site-packages\sqlalchemy\engine\default.py", line 593, in do_execute
    cursor.execute(statement, parameters)
  File "C:\Users\25052288840\Anaconda3\lib\site-packages\pymysql\cursors.py", line 148, in execute
    result = self._query(query)
  File "C:\Users\250522

# Checagens de conformidade

In [21]:

Session = sessionmaker(bind=engine)
session = Session()


In [22]:
rs = session.execute('SELECT count(*) FROM dbmercante.ajna_conformidade;')

In [23]:
rs.scalar()

117954

In [24]:
SQL_CONFORMIDADE = '''
select cod_recinto, count(*), avg(height), std(height),
SUM(if(height < 700, 1, 0)) / count(*) * 100 as height_baixo,
avg(ratio), stddev(ratio),  min(ratio), max(ratio),
 SUM(if(ratio < 1.9, 1, 0)) / count(ratio) * 100 as ratio_baixo
 from ajna_conformidade group by cod_recinto
 '''
df = pd.read_sql(SQL_CONFORMIDADE, engine)

In [25]:
df

,cod_recinto,count(*),avg(height),std(height),height_baixo,avg(ratio),stddev(ratio),min(ratio),max(ratio),ratio_baixo
0,BANDEIRANTES,1437,799.3535,0.4781,0.0000,2.425491,0.699720,1.00,5.73,22.0598
1,BTP1,26234,799.0278,0.1645,0.0000,2.658746,0.507222,0.92,6.02,4.7610
2,BTP2,11636,799.0267,0.1613,0.0000,2.332077,0.443919,0.93,5.53,14.6958
3,CRAGEA,640,879.9313,0.2530,0.0000,2.173406,0.497891,0.45,3.69,10.4688
4,DEICMAR,3699,799.0711,0.2580,0.0000,1.673923,0.443770,0.56,4.24,77.1019
5,ECOPORTO,4495,798.6105,15.4873,0.0667,2.675137,0.559905,0.04,12.28,3.6040
6,EMBRAPORT,15131,799.3144,0.4643,0.0000,4.614342,0.970010,1.30,10.77,0.1917
7,EUDMARCO,1963,212.4116,107.7904,96.5359,3.963515,0.216634,2.49,4.00,0.0000
8,LOCALFRIO,4971,799.0012,0.0401,0.0000,2.201957,0.221659,0.60,3.52,3.1382
9,MARIMEX,5057,799.0295,0.1691,0.0000,2.562500,0.614350,1.05,5.87,10.7178
